Recordemos:

$\mbox{[MAX]} c^T x$

subject to:

$Ax=b$

Dividimos en básicas y no básicas

$c_B x_B + c_N x_N$

sujeto a:

$B x_B + N x_N = b$

operando las restricciones: $x_B = B^{-1} b - B^{-1} N x_N$

y sustituyendo en la función objetivo: $C_B (B^{-1} b - B^{-1} N x_N) + c_N x_N$

Por lo que el modelo se puede escribir como:

$\mbox{[MAX]}~c_B B^{-1} b + (c_N - c_B B^{-1} N) x_N$

sujeto a:

$x_B + B^{-1} N x_N = B^{-1} b$

Entrará la de mayor coste reducido (maximo cRed)
Saldrá la de coeficiente

In [60]:
#introducimos datos
c=[2500 4000 0 0 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]

3x5 Array{Int64,2}:
 1  0  1  0  0
 0  1  0  1  0
 3  5  0  0  1

In [70]:
#seleccionar variable que entra y sale
function selectPivote(costosBasicos,matrizN,rhs)
    #println("matrizN: ",matrizN)
    #println("rhs: ",rhs)
    println("costosBasicos: ",costosBasicos)
    minCol = indmax(costosBasicos) #devuelve posicion con mayor costo reducido
    if costosBasicos[minCol] <= 0.0
        println("Solucion Optima")
        return -1,-1
    end
    #println("\tEntra En Base:",minCol)
    maxVal = typemax(Float64)
    newCol = -1
    for i in 1:size(matrizN,1)
        #println("entro i: ",i," ",rhs[i]," ",matrizN[i,minCol],"\t",maxVal)
        if (matrizN[i,minCol] > 0.0) && (maxVal > rhs[i]/matrizN[i,minCol])
            maxVal = rhs[i]/matrizN[i,minCol]
            newCol = i
        end
    end
    if newCol == -1
        println("Problema no acotado") 
        return -1,-1
    end
    println("entra columna ",minCol," sale ",newCol)
    minCol,newCol
end

#realizar operaciones de una iteración del simplex
function iteracionSimplex(B,cB,N,cN,b,basicas,nobasicas)
    #iteración del simplex
    Binv=B^-1
    obj=cB*Binv*b
    println("obj: ",obj)
    cRed=cN-cB*Binv*N
    #println("costos reducidos:",cRed)
    sol=Binv*b
    #println("solucion:",sol)
    Nprima=Binv*N
    #println("Nprima:",Nprima)
    v1,v2 = selectPivote(cRed,Nprima,sol)
    obj,v1,v2
end

iteracionSimplex (generic function with 1 method)

In [80]:
#construimos base inicial
basicas = Array{Int16}(3)
basicas = [3, 4, 5]
nobasicas = Array{Int16}(2)
nobasicas = [1, 2]
#vamos a dimensionar valores de calculo
B=Array{Float64}(size(A,1),size(A,1))
cB=Array{Float64}(1,size(A,1))
N=Array{Float64}(size(A,1),size(A,2)-size(A,1))
cN=Array{Float64}(1,size(A,2)-size(A,1))

#inicializamos B, cB, N, cN
j=1
for i in basicas
    B[:,j]=A[:,i]
    cB[1,j]=c[1,i]
    j=j+1
end
j=1
for i in nobasicas
    print(i," ")   
    N[:,j]=A[:,i]
    cN[1,j]=c[1,i]
    j=j+1
end    

1 2 

In [72]:
#Box para hacer una iteración
while true
    obj,variableIn,variableOut = iteracionSimplex(B,cB,N,cN,b,basicas,nobasicas)
    if variableIn != -1
        basicas[variableOut] , nobasicas[variableIn] = nobasicas[variableIn] , basicas[variableOut]
        cB[1,variableOut] , cN[1,variableIn] = cN[1,variableIn] , cB[1,variableOut] 
        B[:, variableOut],  N[:,variableIn] = N[:,variableIn] , B[:,variableOut]
        println("obj: ",obj[1]," cB: ", cB, " cN: ",cN," basicas: ",basicas," noBasicas ",nobasicas)
        #println("=========== matriz B ===========")
        #println(B)
        #println("=========== matriz N ===========")
        #println(N)
    else
        println("obj: ",obj[1]," basicas: ",basicas," valores ",B^(-1)*b)
        break
    end
end

obj: [0.0]
costosBasicos: [2500.0 4000.0]
entra columna 2 sale 2
obj: 0.0 cB: [0.0 4000.0 0.0] cN: [2500.0 0.0] basicas: [3,2,5] noBasicas [1,4]
obj: [400000.0]
costosBasicos: [2500.0 -4000.0]
entra columna 1 sale 3
obj: 400000.0 cB: [0.0 4000.0 2500.0] cN: [0.0 0.0] basicas: [3,2,1] noBasicas [5,4]
obj: [608333.3333333334]
costosBasicos: [-833.3333333333333 166.66666666666606]
entra columna 2 sale 1
obj: 608333.3333333334 cB: [0.0 4000.0 2500.0] cN: [0.0 0.0] basicas: [4,2,1] noBasicas [5,3]
obj: [619999.9999999999]
costosBasicos: [-800.0 -99.99999999999955]
Solucion Optima
obj: 619999.9999999999 basicas: [4,2,1] valores [70.0,29.999999999999986,200.0]
